# Sakila DVD Rental Database - EDA

**Projekt:** Lab 1 - SQL Database Analysis  
**Databas:** Sakila (DVD rental store)  
**Verktyg:** DuckDB, Pandas, Python

## Syfte
Utforska Sakila-databasen för att ge insikter till "The Manager" om filmer, kunder, uthyrningar och intäkter.

In [3]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Konfigurera visualiseringar
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

In [4]:
# Funktion för att köra SQL-queries mot DuckDB
def query_sakila(query: str, duckdb_path="../data/sakila.duckdb"):
    """
    Kör en SQL-query mot Sakila DuckDB-databasen.
    
    Parameters:
    -----------
    query : str
        SQL-query att köra
    duckdb_path : str
        Sökväg till DuckDB-databasen
        
    Returns:
    --------
    pandas.DataFrame
        Resultat från queryn
    """
    with duckdb.connect(duckdb_path) as conn:
        df = conn.sql(query).df()
    return df

In [5]:
# Testa att anslutningen fungerar
test_query = """
SELECT COUNT(*) as total_tables
FROM information_schema.tables
WHERE table_schema = 'main'
"""

result = query_sakila(test_query)
print(f"Antal tabeller i databasen: {result['total_tables'].iloc[0]}")

Antal tabeller i databasen: 16


## Task 1: Exploratory Data Analysis (EDA)

### a) Filmer längre än 3 timmar
Vilka filmer är längre än 180 minuter? Visa titel och längd.

In [6]:
# Query för filmer längre än 180 minuter
query_1a = """
SELECT 
    title,
    length
FROM film
WHERE length > 180
ORDER BY length DESC
"""

long_movies = query_sakila(query_1a)
long_movies

,title,length
0,WORST BANGER,185
1,CHICAGO NORTH,185
2,CONTROL ANTHEM,185
3,DARN FORRESTER,185
4,SWEET BROTHERHOOD,185
5,GANGS PRIDE,185
6,HOME PITY,185
7,SOLDIERS EVOLUTION,185
8,POND SEATTLE,185
9,MUSCLE BRIGHT,185


In [7]:
# Visa de 10 längsta filmerna
print(f"Totalt antal filmer längre än 180 minuter: {len(long_movies)}")
long_movies.head(10)

Totalt antal filmer längre än 180 minuter: 39


,title,length
0,WORST BANGER,185
1,CHICAGO NORTH,185
2,CONTROL ANTHEM,185
3,DARN FORRESTER,185
4,SWEET BROTHERHOOD,185
5,GANGS PRIDE,185
6,HOME PITY,185
7,SOLDIERS EVOLUTION,185
8,POND SEATTLE,185
9,MUSCLE BRIGHT,185


**Observation:** Det finns 39 filmer som är längre än 3 timmar. Flera filmer delar förstaplats med 185 minuter, inklusive "WORST BANGER", "CHICAGO NORTH", och "CONTROL ANTHEM".

### b) Filmer med "love" i titeln
Vilka filmer har ordet "love" i sin titel? Visa title, rating, length och description.

In [8]:
# Query för filmer med "love" i titeln
query_1b = """
SELECT 
    title,
    rating,
    length,
    description
FROM film
WHERE title ILIKE '%love%'
ORDER BY title
"""

love_movies = query_sakila(query_1b)
print(f"Totalt antal filmer med 'love' i titeln: {len(love_movies)}")
love_movies

Totalt antal filmer med 'love' i titeln: 10


,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
3,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
4,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
5,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
6,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...
7,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
8,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
9,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...


### c) Beskrivande statistik för filmlängd
Beräkna kortaste, genomsnittliga, median och längsta filmlängden.

In [10]:
# Query för beskrivande statistik på length
query_1c = """
SELECT 
    MIN(length) AS shortest,
    ROUND(AVG(length), 2) AS average,
    MEDIAN(length) AS median,
    MAX(length) AS longest
FROM film
"""

length_stats = query_sakila(query_1c)
length_stats

,shortest,average,median,longest
0,46,115.27,114.0,185


**Observation:** Filmerna varierar mellan 46 och 185 minuter. Genomsnittlig filmlängd är ca 115 minuter (nästan 2 timmar), och median är 114 minuter, vilket visar en ganska symmetrisk fördelning.

### d) Dyraste filmerna att hyra per dag
Vilka är de 10 dyraste filmerna att hyra per dag? (rental_rate / rental_duration)

In [11]:
# Query för dyraste filmer per dag
query_1d = """
SELECT 
    title,
    rental_rate,
    rental_duration,
    ROUND(rental_rate / rental_duration, 2) AS cost_per_day
FROM film
ORDER BY cost_per_day DESC
LIMIT 10
"""

expensive_movies = query_sakila(query_1d)
expensive_movies

,title,rental_rate,rental_duration,cost_per_day
0,ACE GOLDFINGER,4.99,3,1.66
1,AMERICAN CIRCUS,4.99,3,1.66
2,AUTUMN CROW,4.99,3,1.66
3,BACKLASH UNDEFEATED,4.99,3,1.66
4,BEAST HUNCHBACK,4.99,3,1.66
5,BEHAVIOR RUNAWAY,4.99,3,1.66
6,BILKO ANONYMOUS,4.99,3,1.66
7,CARIBBEAN LIBERTY,4.99,3,1.66
8,CASPER DRAGONFLY,4.99,3,1.66
9,CASUALTIES ENCINO,4.99,3,1.66


**Observation:** De 10 dyraste filmerna att hyra per dag kostar alla 1.66 per dag. Detta indikerar att rental_rate och rental_duration är proportionella för toppfilmerna.

### e) Skådespelare som spelat i flest filmer
Vilka är top 10 skådespelarna med flest filmer? Visa skådespelarens namn och antal filmer.

In [12]:
# Query för skådespelare med flest filmer
query_1e = """
SELECT 
    a.first_name || ' ' || a.last_name AS actor_name,
    COUNT(fa.film_id) AS number_of_films
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
GROUP BY a.actor_id, a.first_name, a.last_name
ORDER BY number_of_films DESC
LIMIT 10
"""

top_actors = query_sakila(query_1e)
top_actors

,actor_name,number_of_films
0,GINA DEGENERES,42
1,WALTER TORN,41
2,MARY KEITEL,40
3,MATTHEW CARREY,39
4,SANDRA KILMER,37
5,SCARLETT DAMON,36
6,VAL BOLGER,35
7,UMA WOOD,35
8,VIVIEN BASINGER,35
9,ANGELA WITHERSPOON,35
